In [53]:
import math
import sys
import ezdxf
from shapely.geometry import Point, LineString
from numpy import arctan2, dot, array
from app.main.constants import *
from app.main.models.wall import Wall
from app.main.services.extraction_helper import ExtractionHelper

In [9]:
def center_of_line(x_1, x_2, y_1, y_2):
        x_coordinate = x_1 - 0.5 * (x_1 - x_2)
        y_coordinate = y_1 - 0.5 * (y_1 - y_2)
        return tuple((x_coordinate, y_coordinate))

_____
## Determination of orientation with compass

"normal" determination with compass
___

In [10]:
def get_north_orientation(file_path):
        # read dxf file
        dxf_compass_EG = ezdxf.readfile(file_path)

        # getting paperspace layout by name (bestandplan-compass)
        psp = dxf_compass_EG.layout(NUDATA_COMPASS_LAYOUT)

        # extract the entire compass (all lines of layer 0)
        polylines = [p for p in psp.query('LWPolyline [layer == "0"]')]

        # extract lines of compass
        if len(polylines) == 2:
            listOfPoints = []
            for line in polylines:
                listOfPoints.append((line.get_points()))

            if listOfPoints[0][0] == listOfPoints[1][0]:
                yaxis_endpoint = tuple(
                    (listOfPoints[0][0][0], listOfPoints[0][0][1]))
                yaxis_startpoint = []

                x1 = listOfPoints[0][1][0]
                x2 = listOfPoints[1][1][0]
                y1 = listOfPoints[0][1][1]
                y2 = listOfPoints[1][1][1]

                # possilbe rotations of the compass
                if x1 > x2 and y1 > y2:
                    yaxis_startpoint = center_of_line(
                        x1, x2, y1, y2)
                elif x1 > x2 and y2 > y1:
                    yaxis_startpoint = center_of_line(
                        x1, x2, y2, y1)
                elif x2 > x1 and y1 > y2:
                    yaxis_startpoint = center_of_line(
                        x2, x1, y1, y2)
                elif x2 > x1 and y2 > y1:
                    yaxis_startpoint = center_of_line(
                        x2, x1, y2, y1)

                yaxis_x = yaxis_endpoint[0] - yaxis_startpoint[0]
                yaxis_y = yaxis_endpoint[1] - yaxis_startpoint[1]

                origin = yaxis_startpoint
                xaxis = array([-yaxis_x, yaxis_y, 0.0])
                yaxis = array([-yaxis_y, -yaxis_x, 0.0])

                return origin, xaxis, yaxis

In [12]:
file_path = '../../media/Einfache_Plaene_Compass/R-Bau_D_EG.dxf'
origin, xaxis, yaxis = get_north_orientation(file_path)

In [13]:
print("origin: ", origin)
print("xaxis: ", xaxis)
print("yaxis: ", yaxis)


origin:  (983.3369163510645, 52.603760215210514)
xaxis:  [-14.86406844 -11.28115908   0.        ]
yaxis:  [ 11.28115908 -14.86406844   0.        ]


In [14]:
def full_angle_between(v1, v2):
        dot = v1[0]*v2[0]+v1[1]*v2[1]
        det = v1[0]*v2[1]-v2[0]*v1[1]

        angle = math.degrees(arctan2(det, dot))
        if angle < 0:
            angle = 360-abs(angle)

        return angle

In [15]:
print(full_angle_between(xaxis, yaxis))

90.0


___
## Determination of orientation with angle
### --------- Idea --------
* Construct perpendicular line (points perpentcular to the top)
* rotate this perpendicular line by a given angle alpha
* the new line is the new yaxis
* continue with same logic for processing yaxis
___

In [11]:

line_1 = LineString([Point(0,0), Point(1,0)])
line_1_middle = center_of_line(0,1,0,0)

line_2 = LineString([Point(0,1), Point(1,1)])
line_2_middle = center_of_line(0,1,1,1)

x_1 = float(line_1_middle[0])
x_2 = float(line_2_middle[0])
y_1 = float(line_1_middle[1])
y_2 = float(line_2_middle[1])

perpendicular_line = array([0, 0, 0], dtype=float)
perpendicular_line[0] = x_2-x_1
perpendicular_line[1] = y_2-y_1

print(perpendicular_line)

[0. 1. 0.]


In [12]:
def get_north_orientation_by_angle(angle: float):
    '''
    Description: This method uses perpendicular line and a rotation matrix (generate_rotation_matrix()) to determine a north oriented yaxis.
    Params: angle: float 
    Return: yaxis: numpy.ndarray
    '''

    alpha = math.radians(angle)
    perpendicular_line = array([0, 1, 0])

    # Rotation axis (z-axis)
    axis = array([0, 0, 1])
    yaxis = dot(generate_rotation_matrix(axis, alpha), perpendicular_line)
    return yaxis

In [25]:
def generate_rotation_matrix(axis, alpha):
    '''
    Description: This method utilizes Euler-Rodrigues formular to generate a rotation matrix. Rotation about axis by alpha (in radians).
    Params: angle: float 
    Return: numpy.ndarray
    '''
    
    axis = axis / math.sqrt(dot(axis, axis))
    a = math.cos(alpha / 2.0)
    b, c, d = -axis * math.sin(alpha / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

In [26]:
yaxis_new = get_north_orientation_by_angle(280)
print(yaxis_new)
print(type(yaxis_new))

[0.98480775 0.17364818 0.        ]
<class 'numpy.ndarray'>


In [27]:
print(round(full_angle_between(perpendicular_line, yaxis_new), 4))

280.0


In [64]:
COMPASS_ORIENTATION_PRIORITIZED = False
print(COMPASS_ORIENTATION_PRIORITIZED)

False


In [65]:
calculate_orientation = False
yaxis = None

if COMPASS_ORIENTATION_PRIORITIZED:
    try:
        yaxis = "Compass Orientation"
        calculate_orientation = True
    except Exception:
        pass

        try:
            yaxis = "Angle Orientation"
            calculate_orientation = True
        except KeyError:
            calculate_orientation = False
            

else:
    try:
        yaxis = "Angle Orientation"
        calculate_orientation = True
    except Exception:
        pass
    
        try:
            yaxis = "Compass Orientation"
            calculate_orientation = True
        except KeyError:
            calculate_orientation = False

In [66]:
print(calculate_orientation)
print(yaxis)

True
Angle Orientation
